In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading Dataset**

In [2]:
df_x = pd.read_csv("world_owid.csv", header=0)
df_x.head()

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
0,1750,NaN,9.351,9.351,NaN,NaN,NaN
1,1751,NaN,9.351,9.351,NaN,NaN,NaN
2,1752,NaN,9.354,9.354,NaN,NaN,NaN
3,1753,NaN,9.354,9.354,NaN,NaN,NaN
4,1754,NaN,9.358,9.358,NaN,NaN,NaN


# **Preprocessing Data**

In [3]:

df_x["year"] = pd.to_datetime(df_x["year"], format="%Y")
df_x.index.freq = 'A'

In [4]:
df_x.info()
df = df_x.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         272 non-null    datetime64[ns]
 1   cement_co2   142 non-null    float64       
 2   co2          272 non-null    float64       
 3   coal_co2     272 non-null    float64       
 4   flaring_co2  72 non-null     float64       
 5   gas_co2      140 non-null    float64       
 6   oil_co2      167 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 15.0 KB


In [5]:
test_dates = df_x["year"][130:]
print(test_dates.shape)
#print(df.head())

(142,)


In [6]:
df.tail()

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
267,2017-01-01,1507.923,36096.738,14506.974,391.992,7144.928,12242.628
268,2018-01-01,1569.218,36826.508,14746.831,412.116,7529.847,12266.017
269,2019-01-01,1617.507,37082.559,14725.978,439.254,7647.528,12345.653
270,2020-01-01,1637.537,35264.086,14174.564,407.584,7556.290,11191.809
271,2021-01-01,1672.592,37123.852,14979.598,416.526,7921.830,11837.159


Slicing of Dataframe for co2 and Coal Emissions and data from 1880 onwards

In [7]:
co2_df = df[['year','co2']].loc[(df['year'] >= '1880-01-01')]
#coal_df = df[['year','coal_co2']].loc[(df['year'] >= '1880-01-01')]


In [8]:
#coal_train_size = int(len(log_coal_df) * 0.9)
#coal_train, coal_test = [0:coal_train_size], X[coal_train_size:len(X)]


co2_data = co2_df[['year','co2']].set_index("year").copy()
#co2_train_size = int(len(co2_data) - 10)

In [9]:
co2_train, co2_val, co2_test = co2_data[0:121], co2_data[121:132], co2_data[132:]

In [10]:
co2_val

,co2
year,
2001-01-01,25668.049
2002-01-01,26281.037
2003-01-01,27651.596
2004-01-01,28636.695
2005-01-01,29614.602
2006-01-01,30593.117
2007-01-01,31506.789
2008-01-01,32085.836
2009-01-01,31564.031


## MAPE

In [11]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

# **2 ARIMA**

# Normally you don't need to difference your target values because ARIMA support differencing (d), however I applied differencing ad Log Transform without changing for ARIMA since I already used them for AR - Autoregressive Model above.

In [13]:
co2_val

,co2
year,
2001-01-01,25668.049
2002-01-01,26281.037
2003-01-01,27651.596
2004-01-01,28636.695
2005-01-01,29614.602
2006-01-01,30593.117
2007-01-01,31506.789
2008-01-01,32085.836
2009-01-01,31564.031


In [14]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np



# Define a list of possible values for p, d, and q
p_values = range(0, 20)  # You can adjust the range based on your requirements
d_values = [ 0,1,2,3,4,5]  # You can adjust the range based on your requirements
q_values = range(0, 15)  # You can adjust the range based on your requirements

mae_list = []
# starting point of mae, you amy change with a number that you don't expect as a result, and I just use starting point as 10000 which is really high MAE
best_mae = 10000 # starting point
best_order_ = []

# Try different combinations of p, d, and q
for p in p_values:
    for d in d_values:
        for q in q_values:


          try:
            # Fit ARIMA model with the current parameters
            model = ARIMA(co2_train, order=(p, d, q))
            model_fit = model.fit()

            # Make predictions on the validation set
            forecast_ = model_fit.forecast(steps=len(co2_val))

            # adjust in a reversing differencing function
            #adj_arima_forecast=reverseDiffer(forecast_, co2_df, co2_test)

            # Calculate Mean Absolute Error (MAE) as the evaluation metric
            mae = mean_absolute_error(forecast_, co2_val)
            mae_list.append(mae)
            best_order_.append((p,d,q))
          except Exception as e:
            # Skip the combination if the model fitting fails
            continue


Görüntülenen çıkış son 5000 satıra kısaltıldı.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/stats

In [15]:
# find the index and value of minumum MAE and its parameters (p,d,q)
min_mae_index = mae_list.index(min(mae_list))
min_best_order =best_order_[min_mae_index]



In [16]:
min_best_order

(0, 4, 2)

In [17]:
# Fit the best ARIMA model with the selected order parameters
best_model = ARIMA(co2_train, order=(min_best_order[0], min_best_order[1], min_best_order[2]))
best_model_fit = best_model.fit()

# Print the summary of the best ARIMA model
print(best_model_fit.summary())

# Make predictions on the test set using the best ARIMA model
forecast_arima = best_model_fit.forecast(steps=10)

## adjust in a reversing differencing function
#arima_forecast_df = reverseDiffer(forecast_arima, co2_df, co2_test)

                               SARIMAX Results                                
Dep. Variable:                    co2   No. Observations:                  121
Model:                 ARIMA(0, 4, 2)   Log Likelihood                -868.857
Date:                Sun, 27 Aug 2023   AIC                           1743.714
Time:                        13:43:03   BIC                           1752.000
Sample:                    01-01-1880   HQIC                          1747.078
                         - 01-01-2000                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -1.9999      0.100    -20.011      0.000      -2.196      -1.804
ma.L2          0.9999      0.100      9.978      0.000       0.803       1.196
sigma2      1.467e+05   1.36e-06   1.08e+11      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)


In [18]:
forecast_arima

2001-01-01    26190.573011
2002-01-01    26943.414661
2003-01-01    27712.323576
2004-01-01    28497.475384
2005-01-01    29299.045711
2006-01-01    30117.210184
2007-01-01    30952.144431
2008-01-01    31804.024078
2009-01-01    32673.024753
2010-01-01    33559.322083
Freq: AS-JAN, Name: predicted_mean, dtype: float64

In [19]:
co2_val[:10]

,co2
year,
2001-01-01,25668.049
2002-01-01,26281.037
2003-01-01,27651.596
2004-01-01,28636.695
2005-01-01,29614.602
2006-01-01,30593.117
2007-01-01,31506.789
2008-01-01,32085.836
2009-01-01,31564.031


In [21]:
arima_mae = mean_absolute_error(forecast_arima,co2_val[:10])
print(f"MAE : {arima_mae:3f}")

MAE : 431.673630


In [22]:
mape_score = mean_absolute_percentage_error(forecast_arima,co2_val[:10])
print(f"MAPE: {mape_score:.2f}%")

MAPE: 9.40%


In [24]:
r2_score(forecast_arima,co2_val[:10])

0.9509054843096969

In [23]:
result_arima = pd.concat(forecast_arima,co2_val[:10], axis=1)
result_arima

<ipython-input-23-f23ebc3489fa>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  result_arima = pd.concat(forecast_arima,co2_val[:10], axis=1)


TypeError: ignored

In [ ]:
#

# **TEST**

In [25]:
# Fit the best ARIMA model with the selected order parameters
best_model_ = ARIMA(co2_test, order=(0, 4, 2))
best_model_fit_ = best_model_.fit()

# Print the summary of the best ARIMA model
print(best_model_fit_.summary())

# Make predictions on the test set using the best ARIMA model
forecast_arima_ = best_model_fit.forecast(steps=10)

## adjust in a reversing differencing function
#arima_forecast_df = reverseDiffer(forecast_arima, co2_df, co2_test)

                               SARIMAX Results                                
Dep. Variable:                    co2   No. Observations:                   10
Model:                 ARIMA(0, 4, 2)   Log Likelihood                 -55.685
Date:                Sun, 27 Aug 2023   AIC                            117.370
Time:                        14:08:52   BIC                            116.746
Sample:                    01-01-2012   HQIC                           114.870
                         - 01-01-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.9734      0.997     -0.976      0.329      -2.927       0.980
ma.L2          0.1631      1.112      0.147      0.883      -2.016       2.343
sigma2      6.326e+06   6.26e+06      1.011      0.3

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [28]:
arima_mae_ = mean_absolute_error(forecast_arima_,co2_test)
print(f"MAE : {arima_mae_:3f}")

MAE : 6163.095013


In [29]:
mape_score_ = mean_absolute_percentage_error(forecast_arima_,co2_test)
print(f"MAPE: {mape_score_:.2f}%")

MAPE: 21.46%


In [31]:
r2_score(forecast_arima_,co2_test)

-6.51806352645461